# Kanten aufgrund von "Freundschaftsbeziehungen"

# Facebook Freunde Scrapen
- Eine super Datenquelle um Netzwerke zu verstehen sind die eigenen Freunde. 
- Diese lassen sich von Facebook scrapen, da Facebook natürlich keinen Export erlaubt - d'oh. 

In [1]:
#!pip install selenium
#!pip3 install webdriver-manager
#!pip install lxml

In [2]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json
import csv
import re

In [4]:
#s = webdriver.Chrome('/usr/local/bin/chromedriver')
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

https://selenium-python.readthedocs.io/api.html#selenium.webdriver.remote.webdriver.WebDriver.find_element_by_link_text- 

In [73]:
MY_SLUG = "thomas.plotkowiak"
s = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [126]:
def login(s):
    print("Logging into FB")
    s.get("http://facebook.com/login")
    logged_in = False
    while logged_in == False:
        try:
            if s.find_element("xpath","//*[contains(text(),'Stories')]").text == "Stories":
                logged_in = True
        except:
            time.sleep(1)
            print("Finish login first")

In [127]:
def scrape_friends(s):
    login(s)
    #Scrape my friends
    s.get("https://www.facebook.com/%s/friends" % MY_SLUG)
    for i in range(30):
        print("scrolling...and...waiting..%s" % i)
        s.execute_script('window.scroll(0,100000);')
        time.sleep(1)
    # get html
    elem = s.find_element("xpath","//*")
    source_code = elem.get_attribute("outerHTML")
    soup = BeautifulSoup(source_code, 'lxml')
    links = soup.select("a")
    print("Scraping {%s} links" % len(links))
    with open('friends.csv', mode='w') as myfile:
        mycsv = csv.writer(myfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        mycsv.writerow(["Friend Name"])
        friends = []
        for link in links:
            try:
                friend_handle = re.search("https:\/\/www.facebook.com\/([a-zA-Z.0-9]*)",link["href"])[1]
                friends.append(friend_handle)
            except:
                pass
                #print("Problems obtaining %s" % link)
        friends = list(set(friends))
        for friend in friends:
            mycsv.writerow([friend])

In [128]:
scrape_friends(s)

Logging into FB
scrolling...and...waiting..0
scrolling...and...waiting..1
scrolling...and...waiting..2
scrolling...and...waiting..3
scrolling...and...waiting..4
scrolling...and...waiting..5
scrolling...and...waiting..6
scrolling...and...waiting..7
scrolling...and...waiting..8
scrolling...and...waiting..9
scrolling...and...waiting..10
scrolling...and...waiting..11
scrolling...and...waiting..12
scrolling...and...waiting..13
scrolling...and...waiting..14
scrolling...and...waiting..15
scrolling...and...waiting..16
scrolling...and...waiting..17
scrolling...and...waiting..18
scrolling...and...waiting..19
scrolling...and...waiting..20
scrolling...and...waiting..21
scrolling...and...waiting..22
scrolling...and...waiting..23
scrolling...and...waiting..24
scrolling...and...waiting..25
scrolling...and...waiting..26
scrolling...and...waiting..27
scrolling...and...waiting..28
scrolling...and...waiting..29
Scraping {932} links


In [129]:
def scrape_mutual_friends(s):
    login(s)
    edgelist = []
    friendslist = []
    
    #Get FriendsIDs
    with open('friends.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for i, row in enumerate(csv_reader):
            if i != 0:
                friendslist.append([row["Friend Name"]])

    for friends in friendslist:
        print("Visiting friend: %s" % friends[0])
        s.get("https://www.facebook.com/%s/friends_mutual" % (friends[0]))
        elem = s.find_element("xpath","//*")
        source_code = elem.get_attribute("outerHTML")
        mutual_friends = re.findall("https:\/\/www.facebook.com\/([a-zA-Z.0-9]*)",source_code)
        mutual_friends = list(set(mutual_friends)-set(["photo","stories","profile.php","photo.php"]))
        print("Found %s mutual friends" % len(mutual_friends))
        for friend in mutual_friends:
            if friends[0] != friend : #exclude myself
                edgelist.append([friends[0],friend])
        
    with open('network.csv', mode='w') as myfile:
        mycsv = csv.writer(myfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        mycsv.writerow(["From","To"])
        for edge in edgelist:
            mycsv.writerow([edge[0],edge[1]])

In [130]:
scrape_mutual_friends(s)

Logging into FB
Visiting friend: africartoons
Found 1 mutual friends
Visiting friend: amber.fruhlingly
Found 2 mutual friends
Visiting friend: susan.sunshine.777
Found 3 mutual friends
Visiting friend: franz.droehnack
Found 9 mutual friends
Visiting friend: petra.stroh.7
Found 3 mutual friends
Visiting friend: kathydelcastillo
Found 9 mutual friends
Visiting friend: dr.jedda.jones
Found 2 mutual friends
Visiting friend: biehl.matt
Found 9 mutual friends
Visiting friend: andrea.bender.79
Found 8 mutual friends
Visiting friend: franziska.ruf.9
Found 1 mutual friends
Visiting friend: negar.rajabi
Found 9 mutual friends
Visiting friend: events
Found 1 mutual friends
Visiting friend: wibke1
Found 9 mutual friends
Visiting friend: jhsieg
Found 2 mutual friends
Visiting friend: mech.kamal
Found 5 mutual friends
Visiting friend: julia.ludwig.927
Found 4 mutual friends
Visiting friend: henning.kleb
Found 2 mutual friends
Visiting friend: markus.franz.58
Found 3 mutual friends
Visiting friend: f

Found 2 mutual friends
Visiting friend: yutmun
Found 9 mutual friends
Visiting friend: thomas.engenhorst
Found 9 mutual friends
Visiting friend: eiichi.kumon
Found 6 mutual friends
Visiting friend: alexander.multhaup
Found 6 mutual friends
Visiting friend: markus.sorg.77
Found 6 mutual friends
Visiting friend: jracle
Found 8 mutual friends
Visiting friend: photo
Found 1 mutual friends
Visiting friend: susanne.murbeth
Found 9 mutual friends
Visiting friend: jangumprecht
Found 9 mutual friends
Visiting friend: malte.probst
Found 9 mutual friends
Visiting friend: patrik.riklin
Found 3 mutual friends
Visiting friend: andrew.dick.3576
Found 3 mutual friends
Visiting friend: jana.bimova.75
Found 4 mutual friends
Visiting friend: jodesser
Found 9 mutual friends
Visiting friend: caroline.schmidt.90
Found 9 mutual friends
Visiting friend: rainer.fuehrs
Found 2 mutual friends
Visiting friend: julia.baumann.1293
Found 9 mutual friends
Visiting friend: jtjendra
Found 5 mutual friends
Visiting frie

In [131]:
import networkx as nx

def create_user_network():
    friendslist = []
    edgelist = []
    
    with open('friends.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for i, row in enumerate(csv_reader):
            if i != 0:
                friendslist.append([row["Friend Name"]])        

    with open('network.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for i, row in enumerate(csv_reader):
            if i != 0:
                edgelist.append([row["From"],row["To"]])
    
    G  = nx.Graph(name="Friends")
                            
    for friend in friendslist:
        G.add_node(friend[0],label=friend[0], size=1)

    for edge in edgelist:
        G.add_edge(edge[0],edge[1],weight=1)
   
    nx.write_gexf(G,"Friends.gexf")

In [132]:
create_user_network()